In [2]:
! pip install imbalanced-learn

In [3]:
import numpy as np
import tqdm
import requests
import pandas as pd
import os
import numpy as np
import random
import json
from collections import Counter
import re
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import math
from tensorflow.keras.callbacks import EarlyStopping
from imblearn.over_sampling import SMOTE
import pandas as pd
import numpy as np
import re
from collections import Counter
import math
import aiohttp
import asyncio


2024-07-12 09:14:23.178368: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-12 09:14:23.618430: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-12 09:14:25.215902: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-12 09:14:28.459803: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [166]:
class ColumnAnalysis:

    def __init__(self):
        self.entity_type_dict = {
            "PERSON": "NE",
            "NORP": "NE",
            "FAC": "NE",
            "ORG": "NE",
            "GPE": "NE",
            "LOC": "NE",
            "PRODUCT": "NE",
            "EVENT": "NE",
            "WORK_OF_ART": "NE",
            "LAW": "NE",
            "LANGUAGE": "NE",
            "DATE": "LIT",
            "TIME": "LIT",
            "PERCENT": "LIT",
            "MONEY": "LIT",
            "QUANTITY": "LIT",
            "ORDINAL": "LIT",
            "CARDINAL": "LIT",
            "URL": "LIT",
            "DESC": "LIT",
            "TOKEN": "NE",
            "INTEGER": "LIT",
            "FLOAT": "LIT",
            "DATETIME": "LIT",
            "ADDRESS": "LIT",
            "EMAIL": "LIT"
        }

        self.LIT_DATATYPE = {
            "DATE": "DATETIME", 
            "TIME": "STRING", 
            "PERCENT": "STRING", 
            "MONEY": "STRING", 
            "QUANTITY": "STRING", 
            "ORDINAL": "NUMBER", 
            "CARDINAL": "NUMBER", 
            "URL": "STRING",
            "DESC": "STRING",
            "TOKEN": "STRING",
            "INTEGER": "NUMBER",
            "FLOAT": "NUMBER",
            "DATETIME": "DATETIME",
            "ADDRESS": "STRING",
            "EMAIL": "STRING",
            "STRING": "STRING"
        }

        self.NE_DATATYPE = ["PERSON", "NORP", "FAC", "ORG", "GPE", "LOC", "PRODUCT", "EVENT", "WORK_OF_ART", "LAW", "LANGUAGE"]
    
    def most_frequent_element(self, input_list):
        counter = Counter(input_list)
        most_common = counter.most_common(1)
        return most_common[0][0] if most_common else None

    def extract_number_features(self, column):
        try:
            col = pd.to_numeric(column, errors='coerce')
            return {
                'min_value': np.min(col),
                'max_value': np.max(col),
                'mean_value': np.mean(col),
                'std_dev': np.std(col),
                'unique_count': len(set(col))
            }
        except Exception as e:
            print(f"Error extracting number features: {e}")
            return {}

    def extract_named_entity_features(self, column):
        lengths = [len(str(entry)) for entry in column]
        features = {
            'average_length': np.mean(lengths) if lengths else 0,
            'min_length': np.min(lengths) if lengths else 0,
            'max_length': np.max(lengths) if lengths else 0,
            'all_caps': sum(1 for entry in column if str(entry).isupper()),
            'capitalized': sum(1 for entry in column if str(entry).istitle()),
            'hyphens': sum(str(entry).count('-') for entry in column),
            'periods': sum(str(entry).count('.') for entry in column),
            'commas': sum(str(entry).count(',') for entry in column)
        }
        return features

    def extract_string_features(self, column):
        lengths = [len(str(entry)) for entry in column]
        features = {
            'average_length': np.mean(lengths) if lengths else 0,
            'min_length': np.min(lengths) if lengths else 0,
            'max_length': np.max(lengths) if lengths else 0,
            'all_caps': sum(1 for entry in column if str(entry).isupper()),
            'capitalized': sum(1 for entry in column if str(entry).istitle()),
            'alphabetic_chars': sum(char.isalpha() for entry in column for char in str(entry)),
            'digit_chars': sum(char.isdigit() for entry in column for char in str(entry)),
            'special_chars': sum(not char.isalnum() for entry in column for char in str(entry))
        }
        return features

    def extract_datetime_features(self, column):
        dates = pd.to_datetime(column, errors='coerce')
        features = {
            'min_date': dates.min(),
            'max_date': dates.max(),
            'year_counts': dates.dt.year.value_counts().to_dict(),
            'month_counts': dates.dt.month.value_counts().to_dict()
        }
        return features

    def extract_url_features(self, column):
        url_pattern = re.compile(r'^(https?|ftp)://[^\s/$.?#].[^\s]*$', re.IGNORECASE)
        lengths = [len(str(entry)) for entry in column]
        features = {
            'average_length': np.mean(lengths) if lengths else 0,
            'min_length': np.min(lengths) if lengths else 0,
            'max_length': np.max(lengths) if lengths else 0,
            'valid_urls': sum(1 for entry in column if re.match(url_pattern, str(entry)))
        }
        return features

    def extract_address_features(self, column):
        address_pattern = re.compile(r'\d+\s+\w+\s+(?:street|st|avenue|ave|road|rd|boulevard|blvd|lane|ln|drive|dr|court|ct|circle|cir|place|pl)\.?\s*\w*', re.IGNORECASE)
        lengths = [len(str(entry)) for entry in column]
        features = {
            'average_length': np.mean(lengths) if lengths else 0,
            'min_length': np.min(lengths) if lengths else 0,
            'max_length': np.max(lengths) if lengths else 0,
            'address_count': sum(1 for entry in column if re.match(address_pattern, str(entry)))
        }
        return features

    def extract_email_features(self, column):
        email_pattern = re.compile(r'^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$', re.IGNORECASE)
        lengths = [len(str(entry)) for entry in column]
        features = {
            'average_length': np.mean(lengths) if lengths else 0,
            'min_length': np.min(lengths) if lengths else 0,
            'max_length': np.max(lengths) if lengths else 0,
            'valid_emails': sum(1 for entry in column if re.match(email_pattern, str(entry)))
        }
        return features
    
    async def fetch_entity(self, session, cell):
        if cell is None or pd.isna(cell):
            return None
        cell = str(cell)
        url = 'https://lamapi.hel.sintef.cloud/lookup/entity-retrieval'
        params = {
            'name': cell,
            'token': 'lamapi_demo_2023',
            'kg': 'wikidata',
            'limit': 1000,
            #'query': f'{{"query": {{"bool": {{"must": [{{"match": {{"name": {{"query": "{cell}", "boost": 2.0}}}}}}]}}}}}}',
            #'sort': [
            #    f'''{{"popularity": {{"order": "desc"}}}}'''
            #]
        }
        async with session.get(url, params=params, ssl=False, timeout=100) as response:
            if response.status == 200:
                return await response.json()
            return None

    async def classify_columns_async(self, df):
        def combine_scores(j_score, ed_score, w1=0.5, w2=0.5):
            return w1 * j_score + w2 * ed_score

        url_pattern = re.compile(r'^(https?|ftp)://[^\s/$.?#].[^\s]*$', re.IGNORECASE)
        email_pattern = re.compile(r'^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$', re.IGNORECASE)
        address_pattern = re.compile(r'\d+\s+\w+\s+(?:street|st|avenue|ave|road|rd|boulevard|blvd|lane|ln|drive|dr|court|ct|circle|cir|place|pl)\.?\s*\w*', re.IGNORECASE)
        datetime_pattern = re.compile(
            r'(?:\d{4}-\d{2}-\d{2})'  # YYYY-MM-DD format
            r'|(?:31(?:\/|-|\.)0?[13578]|1[02](?:\/|-|\.)\d{4})'  # 31 days months
            r'|(?:29|30(?:\/|-|\.)0?[1,3-9]|1[0-2](?:\/|-|\.)\d{4})'  # 29/30 days months
            r'|(?:0?[1-9]|[12]\d|3[01])(?:\/|-|\.)'  # Day
            r'(?:0?[1-9]|1[0-2])(?:\/|-|\.)\d{4}'  # Month
            r'|(?:0?[1-9]|1[0-2])/(?:0?[1-9]|[12]\d|3[01])/(?:\d{2})'  # MM/DD/YY format
            r'|(?:0?[1-9]|1[0-2])/(?:0?[1-9]|[12]\d|3[01])/\d{2}'  # MM/DD/YY format
            r'\b\d{2}/(?:0?[1-9]|[12]\d|3[01])/(?:0?[1-9]|1[0-2])\b'  # YY/DD/MM format
            r'|(?:[01]?\d|2[0-3]):[0-5]\d\.[0-5]\d'  # HH:MM.SS format
            r'|(?:[01]?\d|2[0-3]):[0-5]\d'  # HH:MM format
            r'|(?:[0-5]?\d):[0-5]\d(?:\.\d{1,2})?'  # H:MM or H:MM.S format
            r'|(?:2[0-3]|[01]?\d)h[0-5]?\d(?:m[0-5]?\d(?:\.\d{1,2})?s)?',  # HhMMmSSs format
            re.IGNORECASE
        )

        col_type = []
        feature_list = []

        async with aiohttp.ClientSession() as session:
            for col_name, col_data in df.items():
                type = []
                count_cell = 0

                for cell in col_data:
                    label = None
                    is_number = False
    
                    try:
                        if math.isnan(cell):
                            label = "None"
                    except:
                        pass
                        
                    if isinstance(cell, str):
                        if cell == "NaN" or cell == "nan":
                            label = "None"
                        elif re.match(url_pattern, cell):
                            label = "URL"
                        elif re.match(email_pattern, cell):
                            label = "EMAIL"
                        elif re.match(address_pattern, cell):
                            label = "ADDRESS"
                        elif re.match(datetime_pattern, cell):
                            label = "DATETIME"
                    
                    if label is None:  # if it's none of the types below
                        try:
                            cell_str = str(cell)
                            if ',' in cell_str or '.' in cell_str or '%' in cell_str or '$' in cell_str:
                                cell_str = cell_str.replace('.', '').replace(',', '').replace('%', '').replace('$', '')
                            if len(cell_str) - len(re.findall(r'\d', cell_str)) < 5 and len(re.findall(r'\d', cell_str)) != 0:
                                is_number = True
                        except:
                            pass
                    
                    if is_number:
                        label = "NUMBER"
                    elif isinstance(cell, bool):
                        label = "STRING"
                    elif label != "None" and len(cell.split(" ")) >= 15:
                        label = "NOA"
                    elif label != "None" and len(cell.split(" ")) >= 1 and len(cell) <= 4:
                        label = "STRING"
                    
                    if label is not None:
                        type.append(label)
                    else:
                        if count_cell > 5:
                            type.append("STRING")
                            break  
                        else:                
                            tasks = [self.fetch_entity(session, cell) for cell in col_data if cell is not None and count_cell <= 5]
                            responses = await asyncio.gather(*tasks)
                            
                            for cell, data in zip(col_data, responses):
                                #print(f"{cell}-->{data}")
                                if data and len(data) > 0 and data[0]['NERtype'] != None:
                                    if combine_scores(data[0]['jaccard_score'], data[0]['ed_score']) >= 0.7:
                                        type.append(f"NE_{data[0]['NERtype']}")
                                else:
                                    # if you didn't find a NER type for this i2tem
                                    type.append("STRING")
                                count_cell += 1

                most_common_type = self.most_frequent_element(type)
                col_type.append(most_common_type)

                if most_common_type == "NUMBER":
                    features = self.extract_number_features(col_data)
                elif most_common_type in ['NE_PERS', 'NE_LOC', 'NE_ORG', 'NE_OTHERS']:
                    features = self.extract_named_entity_features(col_data)
                elif most_common_type == "STRING" or most_common_type == "NOA":
                    features = self.extract_string_features(col_data)
                elif most_common_type == "DATETIME":
                    features = self.extract_datetime_features(col_data)
                elif most_common_type == "URL":
                    features = self.extract_url_features(col_data)
                elif most_common_type == "ADDRESS":
                    features = self.extract_address_features(col_data)
                elif most_common_type == "EMAIL":
                    features = self.extract_email_features(col_data)
                else:
                    features = {}

                features['column_name'] = col_name
                features['column_type'] = most_common_type
                feature_list.append(features)

        return feature_list

    def classify_columns(self, df):
        loop = asyncio.get_event_loop()
        return loop.run_until_complete(self.classify_columns_async(df))


In [157]:
async def process_table(column_analysis, table_path, train_df, columns):
    df = pd.read_csv(table_path)
    result = await column_analysis.classify_columns_async(df.iloc[1:10])

    for entry in result:
        row = {col: [entry.get(col, None)] for col in columns}
        train_df = pd.concat([train_df, pd.DataFrame(row)], ignore_index=True)

    return train_df

async def main(tables_path):
    column_analysis = ColumnAnalysis()

    columns = [
        'column_name', 'column_type', 'min_value', 'max_value', 'mean_value', 'std_dev', 'unique_count', 'special_values',
        'average_length', 'min_length', 'max_length', 'all_caps', 'capitalized', 'hyphens', 'periods', 'commas', 'common_prefixes', 'common_suffixes',
        'alphabetic_chars', 'digit_chars', 'special_chars', 'min_date', 'max_date', 'date_range', 'year_counts', 'month_counts',
        'valid_urls', 'address_count', 'valid_emails'
    ]
    
    train_df = pd.DataFrame(columns=columns)

    pattern = r'^\.'

    # Create a list of file paths, excluding files that start with a dot
    table_files = [os.path.join(tables_path, table) for table in os.listdir(tables_path) if not re.match(pattern, table)]

    # Randomly select 5000 tables from table_files
    #table_files = random.sample(table_files, 5000)


    for table_file in tqdm(table_files):
        train_df = await process_table(column_analysis, table_file, train_df, columns)

    return train_df

if __name__ == "__main__":
    #tables_path = "./data/Dataset/Dataset/2T_Round4/tables/"
    #tables_path = "./data/Dataset/Dataset/Round4_2020/tables/"    
    tables_path = "./data/Dataset/Dataset/Round3_2019/tables/"
    train_df = await (main(tables_path))

  0%|          | 0/2161 [00:00<?, ?it/s]/tmp/ipykernel_138/2242469751.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  train_df = pd.concat([train_df, pd.DataFrame(row)], ignore_index=True)
/tmp/ipykernel_138/2242469751.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  train_df = pd.concat([train_df, pd.DataFrame(row)], ignore_index=True)
  0%|          | 1/2161 [00:31<18:39:21, 31.09s/it]


ClientConnectorError: Cannot connect to host lamapi.hel.sintef.cloud:443 ssl:False [Connect call failed ('65.108.140.148', 443)]

In [193]:
output_csv_path = "./2T_train_df.csv"
train_df.to_csv(output_csv_path, index=False)

In [4]:
output_csv_path = "./R4_train_df.csv"
train_df.to_csv(output_csv_path, index=False)

In [ ]:
output_csv_path = "./HT2_train_df.csv"
train_df.to_csv(output_csv_path, index=False)

In [32]:
async def process_table(column_analysis, table_path, train_df, columns):
    df = pd.read_csv(table_path)
    result = await column_analysis.classify_columns_async(df.iloc[1:10])

    for entry in result:
        row = {col: [entry.get(col, None)] for col in columns}
        train_df = pd.concat([train_df, pd.DataFrame(row)], ignore_index=True)

    return train_df

async def main(tables_path):
    column_analysis = ColumnAnalysis()
    columns = [
        'column_name', 'column_type', 'min_value', 'max_value', 'mean_value', 'std_dev', 'unique_count', 'special_values',
        'average_length', 'min_length', 'max_length', 'all_caps', 'capitalized', 'hyphens', 'periods', 'commas', 'common_prefixes', 'common_suffixes',
        'alphabetic_chars', 'digit_chars', 'special_chars', 'min_date', 'max_date', 'date_range', 'year_counts', 'month_counts'
    ]
    train_df = pd.DataFrame(columns=columns)

    pattern = r'^\.'

    # Create a list of file paths, excluding files that start with a dot
    table_files = [os.path.join(tables_path, table) for table in os.listdir(tables_path) if not re.match(pattern, table)]

    for table_file in tqdm(table_files):
        train_df = await process_table(column_analysis, table_file, train_df, columns)

    return train_df

if __name__ == "__main__":
    tables_path = "./data/Dataset/Dataset/Round1_T2D/tables/"
    train_df = await (main(tables_path))

  0%|          | 0/64 [00:21<?, ?it/s]


ClientConnectorError: Cannot connect to host lamapi.hel.sintef.cloud:443 ssl:default [Connect call failed ('65.108.140.148', 443)]

In [ ]:
train_df = pd.read_csv("./R1_train_df.csv")

In [ ]:
train_df[['column_type','average_length', 'min_length', 'max_length', 'all_caps', 'capitalized', 'hyphens',
    'periods', 'commas']][train_df['column_type'].isin(['NE_PERS', 'NE_LOC', 'NE_ORG', 'NE_OTHERS'])]

In [ ]:
####################
# READ THE JSON
#####################

json_file_path = "./data/Round1_T2D_f3_sorted_mentions.json"

# Load the JSON file
with open(json_file_path, "r") as file:
    R1_sorted_mentions = json.load(file)

R1_cea = [item[0]for item in R1_sorted_mentions]

In [ ]:
categories = [
    "Place",
    "PopulatedPlace",
    "City",
    "Country",
    "Region",
    "Mountain",
    "Island",
    "Lake",
    "River",
    "Park",
    "Building",
    "HistoricPlace",
    "Monument",
    "Bridge",
    "Road",
    "Airport",
    "Person",
    "Artist",
    "Athlete",
    "Politician",
    "Scientist",
    "Writer",
    "Actor",
    "Musician",
    "MilitaryPerson",
    "Religious",
    "Royalty",
    "Criminal",
    "Organisation",
    "Company",
    "EducationalInstitution",
    "PoliticalParty",
    "SportsTeam",
    "Non-ProfitOrganisation",
    "GovernmentAgency",
    "ReligiousOrganisation",
    "Band",
    "Library",
    "Museum",
    "Hospital",
    "University",
    "TradeUnion"
]

# Mapping of subtypes to macro classes
mapping = {
    "Place": ["PopulatedPlace", "City", "Country", "Region", "Mountain", "Island", "Lake", "River", "Park", "Building", "HistoricPlace", "Monument", "Bridge", "Road", "Airport"],
    "Person": ["Artist", "Athlete", "Politician", "Scientist", "Writer", "Actor", "Musician", "MilitaryPerson", "Religious", "Royalty", "Criminal"],
    "Organisation": ["Company", "EducationalInstitution", "PoliticalParty", "SportsTeam", "Non-ProfitOrganisation", "GovernmentAgency", "ReligiousOrganisation", "Band"],
    "Institution": ["Library", "Museum", "Hospital", "University", "TradeUnion"]
}

In [ ]:
tables = "./data/Dataset/Dataset/Round1_T2D/tables/"

def count_numbers_in_string(s):
    return len(re.findall(r'\d+', str(s)))

median_lengths = []
median_token_counts = []
average_numeric_counts = []
target = []
columns = []

# Iterate through each table
for table in tqdm(os.listdir(tables)):
    table_file = os.path.join(tables, table)
    table_name = table.split(".")[0]
    df = pd.read_csv(table_file)
    
    for col in df.columns:
        column = df[col].astype(str)
        
        # Calculate median length for the current column
        median_length = column.apply(len).median()
        median_lengths.append(median_length)
        
        # Calculate median token count for the current column
        median_token_count = column.apply(lambda x: len(x.split())).median()
        median_token_counts.append(median_token_count)
        
        # Calculate average count of numeric values in the current column
        total_numeric_count = column.apply(count_numbers_in_string).sum()
        average_numeric_count = total_numeric_count / len(df) if len(df) > 0 else 0
        average_numeric_counts.append(average_numeric_count)

        
        # Check for NE flag
        NE_flag = column.isin(R1_cea).any()
        if NE_flag:
            joined_cells = column.str.cat(sep='-')        
            doc = nlp(joined_cells)
            entities = {"ORG": [], "PERS": [], "LOC": [], "OTHERS": []}

            # Extract entities and classify them
            for ent in doc.ents:
                if ent.label_ == "ORG":
                    entities["ORG"].append(ent.text)
                elif ent.label_ == "PERSON":
                    entities["PERS"].append(ent.text)
                elif ent.label_ == "GPE" or ent.label_ == "FAC":  # GPE (Geopolitical Entity)
                    entities["LOC"].append(ent.text)
                else:
                    entities["OTHERS"].append(ent.text)
            
            # Find the key of the longest entities list
            longest_key = max(entities, key=lambda k: len(entities[k]))
            print(f"{joined_cells} --> The key of the longest list is '{longest_key}'")
                    
            target.append("NE")
        elif median_length - average_numeric_count < 2:
            target.append("lit")
        else:
            target.append("None")
    
    columns.extend(df.columns.tolist())

In [ ]:
####################
# READ THE JSON
#####################

json_file_path = "./data/Round3_2019_sorted_mentions.json"

# Load the JSON file
with open(json_file_path, "r") as file:
    R3_sorted_mentions = json.load(file)

R3_cea = [item[0]for item in R3_sorted_mentions]

In [ ]:
tables = "./data/Dataset/Dataset/Round3_2019/tables/"

def count_numbers_in_string(s):
    return len(re.findall(r'\d+', str(s)))

median_lengths = []
median_token_counts = []
average_numeric_counts = []
target = []
columns = []

# Iterate through each table
for table in tqdm(os.listdir(tables)):
    table_file = os.path.join(tables, table)
    table_name = table.split(".")[0]
    df = pd.read_csv(table_file)
    
    for col in df.columns:
        column = df[col].astype(str)
        
        # Calculate median length for the current column
        median_length = column.apply(len).median()
        median_lengths.append(median_length)
        
        # Calculate median token count for the current column
        median_token_count = column.apply(lambda x: len(x.split())).median()
        median_token_counts.append(median_token_count)
        
        # Calculate average count of numeric values in the current column
        total_numeric_count = column.apply(count_numbers_in_string).sum()
        average_numeric_count = total_numeric_count / len(df) if len(df) > 0 else 0
        average_numeric_counts.append(average_numeric_count)
        
        # Check for NE flag
        NE_flag = column.isin(R3_cea).any()
        if NE_flag:
            target.append("NE")
        elif median_length - average_numeric_count < 2:
            target.append("lit")
        else:
            target.append("None")
    
    columns.extend(df.columns.tolist())

In [ ]:
####################
# READ THE JSON
#####################

json_file_path = "./data/2T_Round4_sorted_mentions.json"

# Load the JSON file
with open(json_file_path, "r") as file:
    R4_sorted_mentions = json.load(file)

R4_2T_cea = [item[0]for item in R4_sorted_mentions]

In [ ]:
tables = "./data/Dataset/Dataset/2T_Round4/tables/"

def count_numbers_in_string(s):
    return len(re.findall(r'\d+', str(s)))

median_lengths = []
median_token_counts = []
average_numeric_counts = []
target = []
columns = []

# Iterate through each table
for table in tqdm(os.listdir(tables)):
    table_file = os.path.join(tables, table)
    table_name = table.split(".")[0]
    df = pd.read_csv(table_file)
    
    for col in df.columns:
        column = df[col].astype(str)
        
        # Calculate median length for the current column
        median_length = column.apply(len).median()
        median_lengths.append(median_length)
        
        # Calculate median token count for the current column
        median_token_count = column.apply(lambda x: len(x.split())).median()
        median_token_counts.append(median_token_count)
        
        # Calculate average count of numeric values in the current column
        total_numeric_count = column.apply(count_numbers_in_string).sum()
        average_numeric_count = total_numeric_count / len(df) if len(df) > 0 else 0
        average_numeric_counts.append(average_numeric_count)
        
        # Check for NE flag
        NE_flag = column.isin(R4_2T_cea).any()
        if NE_flag:
            target.append("NE")
        elif median_length - average_numeric_count < 2:
            target.append("lit")
        else:
            target.append("None")
    
    columns.extend(df.columns.tolist())

In [ ]:
####################
# READ THE JSON
#####################

json_file_path = "./data/Round4_sorted_mentions.json"

# Load the JSON file
with open(json_file_path, "r") as file:
    R4_sorted_mentions = json.load(file)

R4_cea = [item[0]for item in R4_sorted_mentions]

In [ ]:
tables = "./data/Dataset/Dataset/Round4_2020/tables/"

def count_numbers_in_string(s):
    return len(re.findall(r'\d+', str(s)))

median_lengths = []
median_token_counts = []
average_numeric_counts = []
target = []
columns = []

# Iterate through each table
for table in tqdm(os.listdir(tables)):
    table_file = os.path.join(tables, table)
    table_name = table.split(".")[0]
    df = pd.read_csv(table_file)
    
    for col in df.columns:
        column = df[col].astype(str)
        
        # Calculate median length for the current column
        median_length = column.apply(len).median()
        median_lengths.append(median_length)
        
        # Calculate median token count for the current column
        median_token_count = column.apply(lambda x: len(x.split())).median()
        median_token_counts.append(median_token_count)
        
        # Calculate average count of numeric values in the current column
        total_numeric_count = column.apply(count_numbers_in_string).sum()
        average_numeric_count = total_numeric_count / len(df) if len(df) > 0 else 0
        average_numeric_counts.append(average_numeric_count)
        
        # Check for NE flag
        NE_flag = column.isin(R4_cea).any()
        if NE_flag:
            target.append("NE")
        elif median_length - average_numeric_count < 2:
            target.append("lit")
        else:
            target.append("None")
    
    columns.extend(df.columns.tolist())

# Model training

In [104]:
# df1 and df2 are the same

df1 = pd.read_csv("./R1_train_df.csv")
#df2 = pd.read_csv("./R3_train_df.csv")
df3 = pd.read_csv("./R4_train_df.csv")
df4 = pd.read_csv("./HT2_train_df.csv")

# filtering because otherwise the model gets values too high
df3 = df3[(df3['max_value'] <= 1.000000e+10) ]

result = pd.concat([df3, df4, df1], axis=0)
result.drop(['date_range', 'year_counts', 'month_counts'], axis=1, inplace=True)

In [105]:
result

,column_name,column_type,min_value,max_value,mean_value,std_dev,unique_count,special_values,average_length,min_length,...,common_prefixes,common_suffixes,alphabetic_chars,digit_chars,special_chars,min_date,max_date,valid_urls,address_count,valid_emails
1,col1,NUMBER,80521.0000,5.186950e+05,310598.222222,149912.648851,9.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,col2,NUMBER,101.0000,1.426000e+03,500.666667,435.768797,9.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,col1,NUMBER,85.0000,3.810000e+02,205.000000,100.807848,9.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,col2,NUMBER,3.2700,1.254100e+01,6.648000,2.582835,9.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,col1,NUMBER,17.3662,5.411520e+01,33.123111,12.269156,9.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318,Met,NUMBER,890.0000,9.640000e+02,914.333333,25.944385,9.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
319,Grid Ref,NUMBER,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
320,Area,STRING,NaN,NaN,NaN,NaN,NaN,NaN,7.0,7.0,...,NaN,NaN,54.0,0.0,9.0,NaN,NaN,NaN,NaN,NaN
321,?,NUMBER,2.0000,9.000000e+00,5.333333,2.309401,6.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [106]:
# Convert the target variable to numeric
label_encoder = LabelEncoder()

formats = ['%Y-%m-%d', '%Y-%m-%d %H:%M:%S']

for fmt in formats:
    result['min_date'] = pd.to_datetime(result['min_date'], format=fmt, errors='coerce')
    result['max_date'] = pd.to_datetime(result['max_date'], format=fmt, errors='coerce')


result['min_month'] = result['min_date'].dt.month
result['min_year'] = result['min_date'].dt.year
result['max_month'] = result['max_date'].dt.month
result['max_year'] = result['max_date'].dt.year
result = result.dropna(subset=['column_type'])


In [107]:
result.iloc[:, 2:26] = result.iloc[:, 2:26].fillna(-1) 
result.iloc[:, 26:30] = result.iloc[:, 26:30].fillna(0)  # fill the ['min_month', 'min_year', 'max_month', 'max_year'] 

X = result.drop(['max_date', 'min_date', 'column_name', 'column_type'], axis=1)  # Drop the target column from features
y = label_encoder.fit_transform(result['column_type'].values)

# One-hot encode the target variable for multiclass classification
y = to_categorical(y)

In [108]:
result.groupby('column_type').size().reset_index(name='count')

,column_type,count
0,ADDRESS,8
1,DATETIME,229
2,NE_LOC,744
3,NE_ORG,269
4,NE_OTHERS,1650
5,NE_PERS,130
6,NOA,1
7,NUMBER,8660
8,STRING,257


In [109]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Apply SMOTE to the training data
smote = SMOTE(random_state=42, k_neighbors=2)
#X_train, y_train = smote.fit_resample(X_train, y_train)

In [110]:
# Define the model
model = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(16, activation='relu'),
    Dense(y_train.shape[1], activation='softmax')  # Output layer with softmax activation for multiclass classification
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
history = model.fit(X_train, y_train, epochs=30, batch_size=32, validation_split=0.2)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy*100:.2f}%')

Epoch 1/30
239/239 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.5200 - loss: 291905.2500 - val_accuracy: 0.8891 - val_loss: 0.4713
Epoch 2/30
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7462 - loss: 2142891.0000 - val_accuracy: 0.8996 - val_loss: 0.4191
Epoch 3/30
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8026 - loss: 491.2078 - val_accuracy: 0.9048 - val_loss: 0.3792
Epoch 4/30
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8339 - loss: 636576.3125 - val_accuracy: 0.9059 - val_loss: 0.3554
Epoch 5/30
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8525 - loss: 21.6121 - val_accuracy: 0.9064 - val_loss: 0.3343
Epoch 6/30
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8589 - loss: 5.4494 - val_accuracy: 0.9053 - val_loss: 0.3233
Epoch 7/30
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8659 - loss: 410.4388 - val_accuracy: 0.9048 - val_loss: 0.3149
Epoch 8/30
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8695 - loss: 1.69

## New predictions (should be correct)

In [167]:
datetime_strings = [
    "2023-02-15 08:30:00",
    "2023-03-20 18:45:00",
    "2023-04-10 10:00:00",
    "2023-05-05 14:20:00"
]

people = [
    "John Smith",
    "Mary Johnson",
    "James Williams",
    "Patricia Brown",
    "Michael Davis",
    "Jennifer Miller",
    "William Wilson",
    "Linda Moore",
    "David Taylor",
    "Barbara Anderson"
]

cities = [
    "Tokyo lake",
    "New York City lake",
    "Paris lake",
    "London lake",
    "Dubai lake",
    "Singapore",
    "Sydney",
    "Berlin",
    "Hong Kong",
    "Rio de Janeiro"
]


#df = pd.DataFrame({'desc': ["minore di 3 anni" for i in range(0, 10)]})
#df = pd.DataFrame({'desc': datetime_strings})
df = pd.DataFrame({'desc': people})
column_analysis = ColumnAnalysis()
df_feat = await column_analysis.classify_columns_async(df)

columns = [
    'min_value', 'max_value', 'mean_value', 'std_dev', 'unique_count', 'special_values',
    'average_length', 'min_length', 'max_length', 'all_caps', 'capitalized', 'hyphens', 'periods', 'commas', 'common_prefixes', 'common_suffixes',
    'alphabetic_chars', 'digit_chars', 'special_chars', 'valid_urls', 'address_count', 'valid_emails', 'min_date',
    'max_date', 'date_range', 'year_counts', 'month_counts'
]

test_df = pd.DataFrame()

for entry in df_feat:
    row = {col: [entry.get(col, None)] for col in columns}
    test_df = pd.DataFrame(row)


KeyError: 'NERtype'

In [ ]:
df_feat

In [ ]:
# Assuming 'test_df' is your DataFrame name
test_df['min_date'] = pd.to_datetime(test_df['min_date'], format='%Y-%m-%d %H:%M:%S')
test_df['max_date'] = pd.to_datetime(test_df['max_date'], format='%Y-%m-%d %H:%M:%S')
test_df['min_month'] = test_df['min_date'].dt.month
test_df['min_year'] = test_df['min_date'].dt.year
test_df['max_month'] = test_df['max_date'].dt.month
test_df['max_year'] = test_df['max_date'].dt.year



In [ ]:
test_df = test_df.drop(['max_date', 'min_date', 'date_range', 'year_counts', 'month_counts'], axis=1)

In [ ]:
# Fill NaN values in columns 2 to 20 with -1
test_df.iloc[:, 0:22] = test_df.iloc[:, 0:22].fillna(-1.0)

# Fill NaN values in columns 23 to 26 with 0
test_df.iloc[:, 22:27] = test_df.iloc[:, 22:27].fillna(0.0)

test_df = test_df.astype(float)

In [ ]:
#for el in test_df.columns:
#    print(f"{el} --> {test_df.at[0,el]}")

In [ ]:
predictions = model.predict(test_df)

# Convert predictions to class labels if necessary
predicted_classes = predictions.argmax(axis=1)

# Display the predictions

print((predicted_classes))
print(label_encoder.inverse_transform(predicted_classes))